In [ ]:
# Part A: Basic definitions and functions - we start by defining the basic constructs needed
# to interact with the Neo4j database. For Part A, we will establish a connection to a test
# database. Create some nodes and relationships and run some queries.

In [1]:
# There are at least 2 libraries for accessing Neo4j via Python. There is the official
# Neo4j driver provided by the company which can be installed using pip (Python package management)
# and via Anaconda for use with Jupyter: conda install -c conda-forge sqlalchemy neo4j-python-driver
# Currently using 
# See https://neo4j.com/developer/python/#python-driver
from neo4j import GraphDatabase
import requests # Fetch and preview data from API
import json # parse JSON
from itertools import permutations
import pandas as pd

In [2]:
# Establish a connection to the home database
conn = GraphDatabase.driver("<host>:<port>", auth=("neo4j", "neo4j"))

In [3]:
# A simple function that creates a database query session and calls the user
# function to execute. This is using the transaction API which ensure
# atomicity and consistency which are technically not need for these simple examples. 
def fnUpdateQuery(conn, fnQuery, args={}):
    ret = None
    with conn.session() as session:
        ret = session.write_transaction(fnQuery, args)
    
    return ret

In [4]:
# Transa
def fnReadQuery(conn, fnQuery, args={}):
    ret = None
    with conn.session() as session:
        ret = session.read_transaction(fnQuery, args)
    
    return ret

In [5]:
# Reset the database by dropping all the nodes
def fnResetDB(transaction, args):
    return transaction.run("MATCH(n) DETACH DELETE(n)")

In [6]:
# Return all the nodes
def fnGetAllNodes(transaction, args):
    result = transaction.run("MATCH(n) RETURN (n)")
    return [record["n"] for record in result]

In [7]:
# A simple function that create a node for each Person and assigns the first and last name.
# Also adding an auto-generated unique ID, in fact a UUID, to make it easier to query later.
def fnCreatePerson(transaction, args):
    result = transaction.run(
        "CREATE (a:Person {id: apoc.create.uuid(), firstName: $inFirstName, lastName: $inLastName}) RETURN a.id", 
        inFirstName=args["firstName"], inLastName=args["lastName"])
    
    return result.single()[0]

In [8]:
def fnGetIdFromName(transaction, args):
    result = transaction.run("MATCH(n:Person) WHERE n.firstName ='" + args["firstName"] + 
                                 "' AND n.lastName ='" + args["lastName"] + "' RETURN n.id AS id")
    return result.single()[0]

In [9]:
# Creating a generic association function that associates
# one person with one or more others
def fnAssociate(transaction, args):
    personId = args["personId"]
    association = args["association"]
    otherIds = args["otherIds"]
    
    # Create a one-way association by default from the person to the other
    for anotherId in otherIds:
        queryStr="""MATCH (a:Person), (b:Person) WHERE a.id = '{0}' AND b.id = '{1}'
                    CREATE (a)-[r:{2}]->(b);""".format(personId, anotherId, association)
        
        transaction.run(queryStr)
    
        
# This is an example of a one-way associations since the worker is not also the boss of the boss
def fnWorksFor(transaction, args):
    fnAssociate(transaction, {"personId": args["personId"], "association": "works_for",
                                     "otherIds": [args["bossId"]]})
    
    fnAssociate(transaction, {"personId": args["bossId"], "association": "employs",
                                     "otherIds": [args["personId"]]})
    
# This is an example of a two way since obviously a friend of A is also A's friend.    
def fnFriendsWith(transaction, args):
    fnAssociate(transaction, {"personId": args["personId"], "association": "friends_with",
                                 "otherIds": [args["friendId"]]})
    
    fnAssociate(transaction, {"personId": args["friendId"], "association": "friends_with",
                                 "otherIds": [args["personId"]]})

def fnSiblings(transaction, args):
    pairings = list(permutations(args["siblingIds"], 2))
    for pair in pairings:
        fnAssociate(transaction, {"personId": pair[0], "association": "sibling",
                                     "otherIds": [pair[1]]})

In [10]:
def fnDisassociate(transaction, args):
    query="MATCH (a:Person {id: '" + args["personId"] + "'})-[r: " + args["association"] + "]->(b:Person {id: '" + args["otherId"] + "'}) DELETE r"
    
    return transaction.run(query)
    
def fnUnfriend(transaction, args):
    fnDisassociate(transaction, {"personId": args["personId"], "association": "friends_with", 
                                 "otherId": args["friendId"]})
    
    fnDisassociate(transaction, {"personId": args["friendId"], "association": "friends_with", 
                                 "otherId": args["personId"]})

In [118]:
# Match all friends of the given person. Notice the arrow means we're
# returning those friends only once even though the relationship is bi-directional.
def fnGetFriends(transaction, args):
    query="MATCH (a:Person {id: '" + args["personId"] + "'})-[r:friends_with]->(b:Person) RETURN b"
    result = transaction.run(query)
    return [record["b"] for record in result]

# Rank function is not easily obtainable with an emphemeral / read-only query
# So let's do it as a to step function. Get the rank based on current counts.
def fnGetRankByNumFriends(transaction, args):
    query="""MATCH (a:Person)-[r1:friends_with]-(b:Person) 
                WITH a.firstName as fName, (count(b)/2) as fCount
                ORDER BY fCount DESC
                WITH COLLECT(DISTINCT {fCount:fCount}) as c
                unwind range(0, size(c)-1) as r
                return r as rank, c[r]["fCount"] as Friend_Count"""
    result = transaction.run(query)
    return [rank for rank in result]

def fnGetMostFriends(transaction, args):
    ranks = fnGetRankByNumFriends(transaction, args)
    query="""MATCH (a:Person)-[r1:friends_with]-(b:Person) 
                WITH a.firstName as fName, (count(b)/2) as fCount
                ORDER BY fCount DESC
                WHERE fCount={0} RETURN fName""".format(ranks[0]["Friend_Count"])
    result = transaction.run(query)
    return [record["fName"] for record in result]

def fnGetLeastFriends(transaction, args):
    ranks = fnGetRankByNumFriends(transaction, args)
    query="""MATCH (a:Person)-[r1:friends_with]-(b:Person) 
                WITH a.firstName as fName, (count(b)/2) as fCount
                ORDER BY fCount DESC
                WHERE fCount={0} RETURN fName""".format(ranks[-1]["Friend_Count"])
    result = transaction.run(query)
    return [record["fName"] for record in result]
    
def fnGetNoFriends(transaction, args):
    query="MATCH (a:Person) WHERE NOT (a)-[:friends_with]-() RETURN a.firstName"
    result = transaction.run(query)
    return [record["a.firstName"] for record in result]

In [138]:
# Assume id is unique and assigns it to exactly only
def fnSetAge(transaction, args):
    query = "MATCH (a:Person {id:'" + args["personId"] + "'})" + "SET a.age={0} RETURN a".format(args["age"])
    result = transaction.run(query)
    return result.single()[0]

# Let's find friends of friends and friends of siblings. Using union to combine all the relationships
# but because this can result in duplicates, especially due to two-way relationships let's explicit 
# filter out using DISTINCT
def fnRecommendFriends(transaction, args):
    query = "MATCH(a:Person {id:'" + args["personId"] + """'})-[r1:friends_with]-()-[r2:friends_with]-(ff1:Person)
               WHERE NOT EXISTS((a)-[r1:friends_with]-(ff1)) AND NOT EXISTS((a)-[r1:sibling]-(ff1))
               RETURN DISTINCT ff1 as ff
               UNION ALL MATCH(b:Person {id:'""" + args["personId"] + """'})-[r1:sibling]-()-[r2:friends_with]-(ff2:Person)
               WHERE NOT EXISTS((b)-[r1:friends_with]-(ff2)) AND NOT EXISTS((b)-[r1:sibling]-(ff2))
               RETURN DISTINCT ff2 as ff"""
    result = transaction.run(query)
    return [record["ff"] for record in result]

# Similar to the function above but we add some "qualifications" (i.e. age)
# Will return over 18 and those without an age. IS NULL and NOT EXISTS are equivalent
def fnRecommendWorkers(transaction, args):
    query = "MATCH(a:Person {id:'" + args["personId"] + """'})-[r1:employs]-(e1:Person)-[r2:friends_with]-(w1:Person)
               WHERE NOT w1.age IS NULL OR w1.age >= 18 
               RETURN DISTINCT w1 as w
               UNION ALL MATCH(b:Person {id:'""" + args["personId"] + """'})-[r1:employs]-(e2:Person)-[r2:sibling]-(w2:Person)
               WHERE NOT w2.age IS NULL OR w2.age >= 18
               RETURN DISTINCT w2 as w"""
    result = transaction.run(query)
    return [record["w"] for record in result]

In [52]:
# Simple utility function that converts a result set into a panda data frame
def fnGetFrameFromResultSet(result):
    return pd.DataFrame([dict(record) for record in result])

In [13]:
# In part A demonstrates basic Neo4j functions using some test data.
# Let's start with a clean slate by resetting the DB
fnUpdateQuery(conn, fnResetDB)

# Insert a couple of people
thomasId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Thomas", "lastName": "McPherson"})
williamId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "William", "lastName": "Jones"})
marcusId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Marcus", "lastName": "Jones"})
jammieId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Jammie", "lastName": "Jones"})
harryId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Harry", "lastName": "Carson"})
johnId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "John", "lastName": "Thompson"})
paulId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Paul", "lastName": "Adams"})
karlId = fnUpdateQuery(conn, fnCreatePerson, {"firstName": "Karl", "lastName": "Thune"})

# Get people records from nodes
people = fnReadQuery(conn, fnGetAllNodes)

# Convert to data frame to make things easier
dfPeople = fnGetFrameFromResultSet(people)
print(dfPeople.head)

<bound method NDFrame.head of   firstName   lastName                                    id
0    Thomas  McPherson  9a710397-85b6-46e6-9156-283f75372d75
1   William      Jones  2515c1ee-cd0f-413a-8607-520b23e59d6b
2    Marcus      Jones  31614ac1-73a5-4bd1-a8c9-82eed472a2c7
3    Jammie      Jones  d273af6a-fdc8-47fb-ac20-af9b0d1710b7
4     Harry     Carson  983e8787-188a-4ef4-97cd-32730ea1d748
5      John   Thompson  a923c7f6-3b1a-4ad9-b101-fa9983c5a561
6      Paul      Adams  d20ddae8-e38a-47f0-b016-783307bc6536
7      Karl      Thune  6375aae0-d2c5-403f-8b2e-8aa7bd151cc7>


In [14]:
# Get the IDs from first and last name, in this case assumed to uniquely identify.
# Note we returned the IDs above on record creation.
thomasId = fnReadQuery(conn, fnGetIdFromName, {"firstName": "Thomas", "lastName": "McPherson"})
williamId = fnReadQuery(conn, fnGetIdFromName, {"firstName": "William", "lastName": "Jones"})

# Let's fetch the IDs by querying 
print("Thomas and William's Ids are %s, %s" % (thomasId, williamId))

# Now let's create some relationships. Since we created the IDs we can use those.
# Let's build some friendships
fnUpdateQuery(conn, fnFriendsWith, {"personId": thomasId, "friendId": williamId})
fnUpdateQuery(conn, fnFriendsWith, {"personId": thomasId, "friendId": jammieId})
fnUpdateQuery(conn, fnFriendsWith, {"personId": harryId, "friendId": thomasId})
fnUpdateQuery(conn, fnFriendsWith, {"personId": paulId, "friendId": harryId})
fnUpdateQuery(conn, fnFriendsWith, {"personId": karlId, "friendId": paulId})
fnUpdateQuery(conn, fnFriendsWith, {"personId": karlId, "friendId": harryId})

# And work relationships
fnUpdateQuery(conn, fnWorksFor, {"personId": jammieId, "bossId": johnId})

# And family
fnUpdateQuery(conn, fnSiblings, {"siblingIds": [williamId, marcusId, jammieId]})

Thomas and William's Ids are 9a710397-85b6-46e6-9156-283f75372d75, 2515c1ee-cd0f-413a-8607-520b23e59d6b


In [89]:
# And now let's do so basic queries that take advantage of graph database.
friends = fnReadQuery(conn, fnGetFriends, {"personId": harryId})
dfFriends = fnGetFrameFromResultSet(friends)
print("Harry's friends are ", ",".join(dfFriends["firstName"]))
print()

# Who has the most friends?
theMost = fnReadQuery(conn, fnGetMostFriends)
print("The most friends: ", theMost)

theLeast = fnReadQuery(conn, fnGetLeastFriends)
print("The least friends: ", theLeast)

Harry's friends are  Karl,Paul,Thomas

The most friends:  ['Thomas', 'Harry']
The least friends:  ['William', 'Jammie']


In [134]:
# Now let's break the connection - i.e. unfriend 
fnUpdateQuery(conn, fnUnfriend, {"personId": karlId, "friendId": harryId})

# And now who has the most friends?
theMost = fnReadQuery(conn, fnGetMostFriends)
print("And now, the most friends: ", theMost)

# No friends at all?
noFriends = fnReadQuery(conn, fnGetNoFriends)
print(noFriends, " have not friends at all.")
print()

# Recommend some friends for Marcus and Jammie?
newFriends = fnReadQuery(conn, fnRecommendFriends, {"personId": marcusId})
dfNewFriends = fnGetFrameFromResultSet(newFriends)
print("Suggest friends for Marcus:\n", dfNewFriends)
print()

newFriends = fnReadQuery(conn, fnRecommendFriends, {"personId": jammieId})
dfNewFriends = fnGetFrameFromResultSet(newFriends)
print("Suggest friends for Jammie:\n", dfNewFriends)

And now, the most friends:  ['Thomas']
['Marcus', 'John']  have not friends at all.

Suggest friends for Marcus:
   firstName   lastName                                    id  age
0    Thomas  McPherson  9a710397-85b6-46e6-9156-283f75372d75   29

Suggest friends for Jammie:
   firstName   lastName                                    id  age
0     Harry     Carson  983e8787-188a-4ef4-97cd-32730ea1d748   26
1    Jammie      Jones  d273af6a-fdc8-47fb-ac20-af9b0d1710b7   24
2   William      Jones  2515c1ee-cd0f-413a-8607-520b23e59d6b   25
3    Thomas  McPherson  9a710397-85b6-46e6-9156-283f75372d75   29


In [139]:
# And let's enhance our people database with some additional information
# by setting the age. 
fnUpdateQuery(conn, fnSetAge, {"personId": jammieId, "age": 24})
fnUpdateQuery(conn, fnSetAge, {"personId": thomasId, "age": 29})
fnUpdateQuery(conn, fnSetAge, {"personId": williamId, "age": 25})
fnUpdateQuery(conn, fnSetAge, {"personId": harryId, "age": 26})
fnUpdateQuery(conn, fnSetAge, {"personId": marcusId, "age": 17})  

newWorkers = fnReadQuery(conn, fnRecommendWorkers, {"personId": johnId})
dfNewWorkers = fnGetFrameFromResultSet(newFriends)
print(dfNewWorkers)

  firstName   lastName                                    id  age
0     Harry     Carson  983e8787-188a-4ef4-97cd-32730ea1d748   26
1    Jammie      Jones  d273af6a-fdc8-47fb-ac20-af9b0d1710b7   24
2   William      Jones  2515c1ee-cd0f-413a-8607-520b23e59d6b   25
3    Thomas  McPherson  9a710397-85b6-46e6-9156-283f75372d75   29


In [ ]:
# Cleanup the connection
conn.close()

In [ ]:
# Part B: Now let's work with a more complex dataset. Neo4j has provided a sandbox environment which
# includes the dataset, cloud db, integrated graphical shell and walkthrough of the data model and queries.
# This can be found here: https://medium.com/neo4j/introducing-the-neo4j-stackoverflow-sandbox-69e3729d01f3

# For this part of the excercise, will be querying the data via REST, importing it into neo4j and running some
# queries.

In [140]:
# Function to fetch 
def fnGetURL(keyword):
    return "https://api.stackexchange.com/2.2/questions?pagesize=100&order=desc&sort=creation&filter=!5-i6Zw8Y)4W7vpy91PMYsKM-k9yzEsSC1_Uxlf&site=stackoverflow&tagged=" + keyword
    

In [141]:
# We're using APOC is a plugin to inject JSON data into Neo4j. APOC is built into many Neo4j environments
# but there are some security restrictions especially with loading files. This works as follows:
#
# This is a fairly complicated query that is actually just a series of chained subqueries that extract tags
# and attribution for each question.
def fnInsertNeo4jQuestions(transaction, args):
    url = fnGetURL(args["keyword"])

    # 1. Convert the list of objects in JSON into rows
    # 2. Assign the title and link as property to each question node
    # 3. For each tag in question, get the tag and merge (assign to) the associated node
    # 4. Extract the answer as a separate node
    # 5. Extract the author of the question
    queryStr = "CALL apoc.load.json('" + url + """') YIELD value 
            UNWIND value.items AS q
            
            MERGE (question:Question {id:q.question_id}) ON CREATE
              SET question.title = q.title, question.share_link = q.share_link,
            question.favorite_count = q.favorite_count
        
            FOREACH (tagName IN q.tags | MERGE (tag:Tag {name:tagName}) MERGE (question)-[:TAGGED]->(tag))
       
            FOREACH (a IN q.answers |
                MERGE (question)<-[:ANSWERS]-(answer:Answer {id:a.answer_id})
                MERGE (answerer:User {id:a.owner.user_id}) ON CREATE SET answerer.display_name = a.owner.display_name
                MERGE (answer)<-[:PROVIDED]-(answerer)
            )
        
            WITH * WHERE NOT q.owner.user_id IS NULL
                MERGE (owner:User {id:q.owner.user_id}) ON CREATE SET owner.display_name = q.owner.display_name
                SET question.title = q.title, question.share_link = q.share_link, 
                    question.favorite_count = q.favorite_count
            """
    result = transaction.run(queryStr)

In [142]:
# Fetch all Neo4j related questions. This is done via the helper function
# which formats URL with the search term. In this case, let's get some questions on Neo4j.
url = fnGetURL("Neo4j")
print(url)

# Let's preview the JSON data representing the questions and answers
response = requests.get(url)

# Confirm 200 if data was successfully fetched
print(response)

# Print out the 1st example
qaData = json.loads(response.content.decode('utf-8'))["items"]
#print(json.dumps(qaData[0:1], indent=2))

https://api.stackexchange.com/2.2/questions?pagesize=100&order=desc&sort=creation&filter=!5-i6Zw8Y)4W7vpy91PMYsKM-k9yzEsSC1_Uxlf&site=stackoverflow&tagged=Neo4j
<Response [200]>


In [145]:
# Flush the database
fnUpdateQuery(conn, fnResetDB)

ret = fnUpdateQuery(conn, fnGetAllNodes)
print(ret)

[]


In [ ]:
# Use APOC to insert JSON
fnSendQuery(conn, fnInsertNeo4jQuestions, {"keyword": "Neo4j"})